https://www.kaggle.com/c/santander-customer-transaction-prediction

https://www.kaggle.com/roydatascience/santander-transaction-stacking-1-0

https://www.kaggle.com/roydatascience/eda-pca-lgbm-santander-transactions

In [52]:
import numpy as np
import pandas as pd
import datetime
import gc
import time
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier

from sklearn.model_selection import StratifiedKFold, RepeatedKFold, KFold
from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss
from sklearn.impute  import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge, LinearRegression

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE_TSG = 2019
np.random.seed(RANDOM_STATE_TSG)

file_path = 'F:/github_me_repos/Kaggle_code/santander-customer-transaction-prediction/'
print('finished!')

finished!


In [15]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print('finished!')

finished!


In [13]:
df_train = pd.read_csv(file_path + 'data/train.csv')
df_test = pd.read_csv(file_path + 'data/test.csv')

print('finished!')

finished!


In [18]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 78.01 Mb (0.0% reduction)
Mem. usage decreased to 77.82 Mb (74.6% reduction)


In [19]:
df_train.head(10)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.921875,-6.785156,11.906250,5.093750,11.460938,-9.281250,5.117188,18.625000,...,4.433594,3.964844,3.136719,1.691406,18.515625,-2.398438,7.878906,8.562500,12.781250,-1.091797
1,train_1,0,11.500000,-4.148438,13.859375,5.390625,12.359375,7.042969,5.621094,16.531250,...,7.640625,7.722656,2.583984,10.953125,15.429688,2.033203,8.125000,8.789062,18.359375,1.952148
2,train_2,0,8.609375,-2.746094,12.078125,7.894531,10.585938,-9.085938,6.941406,14.617188,...,2.906250,9.789062,1.669922,1.685547,21.609375,3.142578,-6.519531,8.265625,14.718750,0.396484
3,train_3,0,11.062500,-2.152344,8.953125,7.195312,12.585938,-1.835938,5.843750,14.921875,...,4.464844,4.742188,0.717773,1.421875,23.031250,-1.270508,-2.927734,10.289062,17.968750,-9.000000
4,train_4,0,9.835938,-1.483398,12.875000,6.636719,12.273438,2.449219,5.941406,19.250000,...,-1.490234,9.523438,-0.150757,9.195312,13.289062,-1.511719,3.925781,9.500000,18.000000,-8.812500
5,train_5,0,11.476562,-2.318359,12.609375,8.625000,10.960938,3.560547,4.531250,15.226562,...,-6.308594,6.601562,5.292969,0.440186,14.945312,1.031250,-3.625000,9.765625,12.578125,-4.761719
6,train_6,0,11.812500,-0.083191,9.351562,4.292969,11.132812,-8.023438,6.195312,12.078125,...,8.781250,6.453125,3.533203,0.177734,18.328125,0.584473,9.109375,9.117188,10.890625,-3.208984
7,train_7,0,13.554688,-7.988281,13.875000,7.597656,8.656250,0.831055,5.687500,22.328125,...,13.171875,6.550781,3.990234,5.804688,23.140625,-0.377686,4.218750,9.421875,8.664062,3.480469
8,train_8,0,16.109375,2.443359,13.929688,5.632812,8.804688,6.164062,4.453125,10.187500,...,1.429688,14.750000,1.639648,1.417969,14.835938,-1.994141,-1.073242,8.195312,19.515625,4.843750
9,train_9,0,12.507812,1.974609,8.898438,5.449219,13.601562,-16.281250,6.062500,16.843750,...,0.554199,6.316406,1.037109,3.689453,14.835938,0.446777,14.125000,7.914062,16.234375,14.250000


In [20]:
df_test.dtypes
non_numerics = [x for x in df_test.columns if not (df_test[x].dtype == np.float64 or df_test[x].dtype == np.int64)]
print('non_numerics is ', non_numerics)

non_numerics is  ['ID_code', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8', 'var_9', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14', 'var_15', 'var_16', 'var_17', 'var_18', 'var_19', 'var_20', 'var_21', 'var_22', 'var_23', 'var_24', 'var_25', 'var_26', 'var_27', 'var_28', 'var_29', 'var_30', 'var_31', 'var_32', 'var_33', 'var_34', 'var_35', 'var_36', 'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42', 'var_43', 'var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50', 'var_51', 'var_52', 'var_53', 'var_54', 'var_55', 'var_56', 'var_57', 'var_58', 'var_59', 'var_60', 'var_61', 'var_62', 'var_63', 'var_64', 'var_65', 'var_66', 'var_67', 'var_68', 'var_69', 'var_70', 'var_71', 'var_72', 'var_73', 'var_74', 'var_75', 'var_76', 'var_77', 'var_78', 'var_79', 'var_80', 'var_81', 'var_82', 'var_83', 'var_84', 'var_85', 'var_86', 'var_87', 'var_88', 'var_89', 'var_90', 'var_91', 'var_92', 'var_93', 'var_94', 'var_95', 'var_96', 'var_97', '

In [22]:
df_train.shape, df_test.shape

((200000, 202), (200000, 201))

In [23]:
train_y = df_train['target']
train_X = df_train.drop(columns=['ID_code', 'target'])

test_id = df_test['ID_code']
test_X = df_test.drop(columns=['ID_code'])

In [30]:
param_regressor_1 = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.007,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         'random_state': 2019
}


#folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

print('get here!!!')


oof_regressor_full_1 = np.zeros(len(df_train))
predictions_test = np.zeros(len(df_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_X.iloc[trn_idx], label=train_y.iloc[trn_idx])
    val_data = lgb.Dataset(train_X.iloc[val_idx], label=train_y.iloc[val_idx])

    num_round = 10000
#     num_round = 200
    print('start regressor training...')
    clf = lgb.train(param_regressor_1, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_full_1[val_idx] = clf.predict(train_X.iloc[val_idx], num_iteration=clf.best_iteration)    
    predictions_test += clf.predict(test_X, num_iteration=clf.best_iteration) / folds.n_splits
    auc_val = roc_auc_score(train_y.iloc[val_idx], oof_regressor_full_1[val_idx])
    print('current auc val is ', auc_val)
    
print('final rmse val 1 is ', roc_auc_score(train_y, oof_regressor_full_1))

get here!!!
fold n°0
start regressor training...
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.787665	valid_1's auc: 0.758264
[200]	training's auc: 0.827815	valid_1's auc: 0.794824
[300]	training's auc: 0.851109	valid_1's auc: 0.815019
[400]	training's auc: 0.867667	valid_1's auc: 0.829028
[500]	training's auc: 0.879975	valid_1's auc: 0.838648
[600]	training's auc: 0.889676	valid_1's auc: 0.846075
[700]	training's auc: 0.897772	valid_1's auc: 0.85233
[800]	training's auc: 0.904753	valid_1's auc: 0.857541
[900]	training's auc: 0.910599	valid_1's auc: 0.861856
[1000]	training's auc: 0.915604	valid_1's auc: 0.865421
[1100]	training's auc: 0.920101	valid_1's auc: 0.868321
[1200]	training's auc: 0.924015	valid_1's auc: 0.870899
[1300]	training's auc: 0.927593	valid_1's auc: 0.873115
[1400]	training's auc: 0.930894	valid_1's auc: 0.875174
[1500]	training's auc: 0.933856	valid_1's auc: 0.876964
[1600]	training's auc: 0.936638	valid_1's auc: 0.878654
[1

[600]	training's auc: 0.890169	valid_1's auc: 0.848753
[700]	training's auc: 0.897906	valid_1's auc: 0.854262
[800]	training's auc: 0.904424	valid_1's auc: 0.85903
[900]	training's auc: 0.910223	valid_1's auc: 0.863207
[1000]	training's auc: 0.915185	valid_1's auc: 0.866607
[1100]	training's auc: 0.919587	valid_1's auc: 0.869669
[1200]	training's auc: 0.92346	valid_1's auc: 0.872296
[1300]	training's auc: 0.927039	valid_1's auc: 0.874524
[1400]	training's auc: 0.930184	valid_1's auc: 0.876716
[1500]	training's auc: 0.933082	valid_1's auc: 0.878491
[1600]	training's auc: 0.935831	valid_1's auc: 0.880144
[1700]	training's auc: 0.938375	valid_1's auc: 0.881541
[1800]	training's auc: 0.940708	valid_1's auc: 0.88301
[1900]	training's auc: 0.943035	valid_1's auc: 0.884294
[2000]	training's auc: 0.94511	valid_1's auc: 0.88546
[2100]	training's auc: 0.947031	valid_1's auc: 0.88647
[2200]	training's auc: 0.948807	valid_1's auc: 0.887395
[2300]	training's auc: 0.950596	valid_1's auc: 0.888231
[2

[400]	training's auc: 0.86938	valid_1's auc: 0.825511
[500]	training's auc: 0.881373	valid_1's auc: 0.834783
[600]	training's auc: 0.890761	valid_1's auc: 0.841946
[700]	training's auc: 0.898642	valid_1's auc: 0.847942
[800]	training's auc: 0.905328	valid_1's auc: 0.852711
[900]	training's auc: 0.911063	valid_1's auc: 0.856632
[1000]	training's auc: 0.915993	valid_1's auc: 0.860061
[1100]	training's auc: 0.920473	valid_1's auc: 0.863173
[1200]	training's auc: 0.924426	valid_1's auc: 0.865709
[1300]	training's auc: 0.927997	valid_1's auc: 0.868059
[1400]	training's auc: 0.931178	valid_1's auc: 0.87019
[1500]	training's auc: 0.934213	valid_1's auc: 0.872132
[1600]	training's auc: 0.936985	valid_1's auc: 0.873827
[1700]	training's auc: 0.939483	valid_1's auc: 0.875333
[1800]	training's auc: 0.941831	valid_1's auc: 0.876701
[1900]	training's auc: 0.943999	valid_1's auc: 0.877951
[2000]	training's auc: 0.946044	valid_1's auc: 0.878993
[2100]	training's auc: 0.947917	valid_1's auc: 0.880064


In [49]:
train_X.shape, train_y.shape, test_X.shape, type(train_y)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X)):
    print("fold n°{}".format(fold_))
    print('train_X.iloc[trn_idx].shape is ', train_X.iloc[trn_idx].values.shape)
    print('train_X.iloc[val_idx].shape is ', train_X.iloc[val_idx].values.shape)
    
    
#     etr.fit(train_X.iloc[trn_idx].values, train_y.iloc[trn_idx].values)
#     oof_etr[val_idx] = etr.predict_proba(train_X.iloc[val_idx].values)
#     predictions_etr += etr.predict_proba(test_X.values) / folds.n_splits
#     auc_val = roc_auc_score(train_y.iloc[val_idx], oof_etr[val_idx])
#     print('etr current auc_val is ', auc_val)
#     auc_val_lst.append(auc_val)

fold n°0
train_X.iloc[trn_idx].shape is  (160000, 200)
train_X.iloc[val_idx].shape is  (40000, 200)
fold n°1
train_X.iloc[trn_idx].shape is  (160000, 200)
train_X.iloc[val_idx].shape is  (40000, 200)
fold n°2
train_X.iloc[trn_idx].shape is  (160000, 200)
train_X.iloc[val_idx].shape is  (40000, 200)
fold n°3
train_X.iloc[trn_idx].shape is  (160000, 200)
train_X.iloc[val_idx].shape is  (40000, 200)
fold n°4
train_X.iloc[trn_idx].shape is  (160000, 200)
train_X.iloc[val_idx].shape is  (40000, 200)


In [50]:
etr = ExtraTreesClassifier(n_estimators=50, n_jobs=4, min_samples_split=31, 
                          max_depth=9, min_samples_leaf=17, max_features=53, 
                          verbose=200, random_state=1234)

folds = KFold(n_splits=5, shuffle=True, random_state=1234)

oof_etr = np.zeros(len(train_X))
predictions_etr = np.zeros(len(test_X))

auc_val_lst = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X)):
    print("fold n°{}".format(fold_))
    etr.fit(train_X.iloc[trn_idx].values, train_y.iloc[trn_idx].values)
    oof_etr[val_idx] = etr.predict_proba(train_X.iloc[val_idx].values)
#     predictions_etr += etr.predict_proba(test_X.values) / folds.n_splits
    auc_val = roc_auc_score(train_y.iloc[val_idx], oof_etr[val_idx])
    print('etr current auc_val is ', auc_val)
    auc_val_lst.append(auc_val)
    
print('auc_val_lst is ', auc_val_lst)
print('final auc val is ', roc_auc_score(train_y, oof_etr))

fold n°0
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 2 of 50building tree 3 of 50
building tree 4 of 50

building tree 1 of 50
building tree 5 of 50[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    1.1s
building tree 6 of 50[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    1.1s


building tree 7 of 50[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    1.2s

building tree 8 of 50[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:    1.2s

building tree 9 of 50[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.3s

building tree 10 of 50[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:    2.4s

building tree 11 of 50[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:    2.4s

building tree 12 of 50[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:    2.5s

building tree 13 of 50[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:    3.4s

building tree 14 of 50[Parallel(n_jobs=4)]: Done  10 tasks      | elapse

ValueError: shape mismatch: value array of shape (40000,2) could not be broadcast to indexing result of shape (40000,)

In [57]:
current_random_state = 1985

# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp.fit(np.vstack([train_X.values, test_X.values]))
# train_X_new = imp.transform(train_X.values)
# test_X_new = imp.transform(test_X.values)

# scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
# scaler.fit(np.vstack([train_X_new, test_X_new]))
# train_X_new = scaler.transform(train_X_new)
# test_X_new = scaler.transform(test_X_new)

train_X_new = train_X.values
train_X_new = train_X.values
train_y_new = train_y.values

catboost_model = CatBoostClassifier(iterations=300, depth=6, learning_rate=0.008, 
                                    loss_function='CrossEntropy', colsample_bylevel=0.7,
                                    random_state=current_random_state)
folds = KFold(n_splits=5, shuffle=True, random_state=current_random_state)

oof_full_catboost = np.zeros(len(train_X))
predictions_full_catboost = np.zeros(len(test_X))
current_auc_lst = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X_new)):
    print("fold n°{}".format(fold_))
    catboost_model.fit(train_X_new[trn_idx], train_y_new[trn_idx], cat_features=[], 
                       eval_set=(train_X_new[val_idx], train_y_new[val_idx]), 
                       verbose_eval=100, 
                       early_stopping_rounds = 300, 
                       plot=False)
    print('after fit')
    train_val = train_X_new[val_idx]
    print('train_val.shape is ', train_val.shape)
#     oof_full_catboost[val_idx] = catboost_model.predict_proba(train_X_new[val_idx])
    oof_full_catboost[val_idx] = catboost_model.predict_proba(train_val)
    predictions_full_catboost += catboost_model.predict_proba(test_X_new) / folds.n_splits
    auc_val = roc_auc_score(train_y[val_idx], oof_full_catboost[val_idx])
    print('current auc val is ', auc_val)
    current_auc_lst.append(auc_val)

print('current_random_state: ', current_random_state)
print('current_auc_lst catboost: ', current_auc_lst)
print('final auc val catboost: ', roc_auc_score(train_y, oof_full_catboost))

fold n°0
0:	learn: 0.6861482	test: 0.6861357	best: 0.6861357 (0)	total: 285ms	remaining: 1m 25s
100:	learn: 0.3701965	test: 0.3695712	best: 0.3695712 (100)	total: 28.6s	remaining: 56.3s
200:	learn: 0.3133305	test: 0.3131322	best: 0.3131322 (200)	total: 55.7s	remaining: 27.5s
299:	learn: 0.2966700	test: 0.2970676	best: 0.2970676 (299)	total: 1m 21s	remaining: 0us

bestTest = 0.2970676219
bestIteration = 299

after fit
train_val.shape is  (40000, 200)


ValueError: shape mismatch: value array of shape (40000,2) could not be broadcast to indexing result of shape (40000,)

In [31]:
outcome = pd.DataFrame()
outcome['ID_code'] = test_id
outcome['target'] = predictions_test
outcome.to_csv(file_path + '/submission/outcome_8962.csv', index=False)